In [2]:
import pandas as pd
import plotly_express as px

In [3]:
# reading athlete_events as df1
df1 = pd.read_csv("../Data/athlete_events.csv")
#df1.head()

# reading noc_regions as df2, going to merge these
df2 = pd.read_csv("../Data/noc_regions.csv")
#df2.head()
#df1.head(1)

# merging both files into one on NOC
df = pd.merge(df1, df2, on="NOC", how="left")
df.head(2)

# cross country skiing only dataframe
xcs_df = df[df["Sport"] == "Cross Country Skiing"]
xcs_df.head()


# dropping 'ID', 'Name', 'Age', 'Height', 'Weight' and NaN medals
dropped_df = xcs_df.drop(columns=['ID','Name', 'Age', 'Height','Weight']).dropna(subset='Medal')

# dropping duplicates to seperate team events
# going to use 'reusable_df' as a basline dataframe
# 
reusable_df = dropped_df.drop_duplicates()

In [4]:
# TODO: delete this cell
dropped_df.count() # count = 766

Sex       776
Team      776
NOC       776
Games     776
Year      776
Season    776
City      776
Sport     776
Event     776
Medal     776
region    776
notes       0
dtype: int64

In [5]:
# TODO: delete this cell
reusable_df.count() # count = 461

Sex       461
Team      461
NOC       461
Games     461
Year      461
Season    461
City      461
Sport     461
Event     461
Medal     461
region    461
notes       0
dtype: int64

### Men relay 10 km 

In [6]:
# taking 'Cross Country Skiing Men's 4 x 10 kilometres Relay' only to save in a dataframe
df_men_relay = reusable_df[reusable_df["Event"] == "Cross Country Skiing Men's 4 x 10 kilometres Relay"]

# grouping by event + region. new column '0', counts medals. 
# going to rename this one after concat with other mens team event
df_men_relay = df_men_relay.groupby(["Event", "region"])[["Medal"]].value_counts().to_frame().reset_index()

#TODO: sort medals
df_men_relay.head(2)

,Event,region,Medal,0
0,Cross Country Skiing Men's 4 x 10 kilometres R...,Czech Republic,Bronze,2
1,Cross Country Skiing Men's 4 x 10 kilometres R...,Finland,Bronze,6


In [7]:
# same as above but for xcs men's team sprint
df_men_sprint = reusable_df[reusable_df["Event"] == "Cross Country Skiing Men's Team Sprint"]
df_men_sprint = df_men_sprint.groupby(["Event", "region"])[["Medal"]].value_counts().to_frame().reset_index()
df_men_sprint.head(2)

,Event,region,Medal,0
0,Cross Country Skiing Men's Team Sprint,Finland,Gold,1
1,Cross Country Skiing Men's Team Sprint,Germany,Silver,1


### concating df_men_relay and df_men_sprint

In [8]:
frames = [df_men_sprint, df_men_relay]
concat_men_df = pd.concat(frames)
# This df consists of df_men_sprint AND df_men_relay
concat_men_df.head(9)

,Event,region,Medal,0
0,Cross Country Skiing Men's Team Sprint,Finland,Gold,1
1,Cross Country Skiing Men's Team Sprint,Germany,Silver,1
2,Cross Country Skiing Men's Team Sprint,Norway,Gold,1
3,Cross Country Skiing Men's Team Sprint,Norway,Silver,1
4,Cross Country Skiing Men's Team Sprint,Russia,Bronze,2
5,Cross Country Skiing Men's Team Sprint,Russia,Silver,1
6,Cross Country Skiing Men's Team Sprint,Sweden,Bronze,1
7,Cross Country Skiing Men's Team Sprint,Sweden,Gold,1
0,Cross Country Skiing Men's 4 x 10 kilometres R...,Czech Republic,Bronze,2


## sorting medals to get a nicer plot
* and renaming column '0' -> 'Amount'

In [9]:
concat_men_df = concat_men_df.rename({0:'Amount'}, axis=1)

# found this on stackoverflow:
# https://stackoverflow.com/questions/52784410/sort-column-in-pandas-dataframe-by-specific-order

concat_men_df.Medal = pd.Categorical(concat_men_df.Medal,categories=['Bronze', 'Silver', 'Gold'])
concat_men_df = concat_men_df.sort_values('Medal')
concat_men_df

,Event,region,Medal,Amount
18,Cross Country Skiing Men's 4 x 10 kilometres R...,Switzerland,Bronze,1
16,Cross Country Skiing Men's 4 x 10 kilometres R...,Sweden,Bronze,4
13,Cross Country Skiing Men's 4 x 10 kilometres R...,Russia,Bronze,3
4,Cross Country Skiing Men's Team Sprint,Russia,Bronze,2
11,Cross Country Skiing Men's 4 x 10 kilometres R...,Norway,Bronze,1
6,Cross Country Skiing Men's Team Sprint,Sweden,Bronze,1
0,Cross Country Skiing Men's 4 x 10 kilometres R...,Czech Republic,Bronze,2
1,Cross Country Skiing Men's 4 x 10 kilometres R...,Finland,Bronze,6
6,Cross Country Skiing Men's 4 x 10 kilometres R...,Germany,Bronze,1
4,Cross Country Skiing Men's 4 x 10 kilometres R...,France,Bronze,1


## plotting the concated df

In [10]:
fig = px.histogram(
    concat_men_df,
    x="region",
    y="Amount",
    color="Medal",
    labels={"Sport": "Sport", "0": "medals", "region": "Country"},
    barmode="group",
    title="Men's team cross country skiing medals",
    text_auto = True,
    #pattern_shape = "Medal", pattern_shape_sequence=["-", "x", "+"] # creates a pattern on each bar. kinda ugly
    color_discrete_sequence=[px.colors.qualitative.Dark2[6],px.colors.qualitative.Dark2[7],px.colors.qualitative.Dark2[5]]
)
# color_bars = {"Silver": "silver", "Bronze": "bronze", "Gold": "gold"}
# fig.update_traces(marker_color=color_bars)
fig.show()

In [11]:
#TODO: Create all 3 plots
#TODO: subplots with px 3x1. Visa enbart länder med 
#TODO: Live adjustable subplot-width?

___
# Womens XCS team sports

Cross Country Skiing Women's 3 x 5 kilometres Relay

In [12]:
df_women_relay = reusable_df[reusable_df["Event"] == "Cross Country Skiing Women's 3 x 5 kilometres Relay"]
df_women_relay = df_women_relay.groupby(["Event", "region"])[["Medal"]].value_counts().to_frame().reset_index()
#df_women_relay

Cross Country Skiing Women's 4 x 5 kilometres Relay



In [13]:
df_women_4x_relay = reusable_df[reusable_df["Event"] == "Cross Country Skiing Women's 3 x 5 kilometres Relay"]
df_women_4x_relay = df_women_4x_relay.groupby(["Event", "region"])[["Medal"]].value_counts().to_frame().reset_index()

Cross Country Skiing Women's Team Sprint



In [14]:
df_women_sprint = reusable_df[reusable_df["Event"] == "Cross Country Skiing Women's Team Sprint"]
df_women_sprint = df_women_sprint.groupby(["Event", "region"])[["Medal"]].value_counts().to_frame().reset_index()


concating womens team sport into one dataframe

In [15]:
frames = [df_women_relay, df_women_4x_relay, df_women_sprint]
concat_women_team = pd.concat(frames)
#concat_women_team

In [16]:
concat_women_team = concat_women_team.rename({0:'Amount'}, axis=1)

concat_women_team.Medal = pd.Categorical(concat_women_team.Medal,categories=['Bronze', 'Silver', 'Gold'])
concat_women_team = concat_women_team.sort_values('Medal')
#concat_women_team

## Plotting concated women team

In [17]:
fig = px.histogram(
    concat_women_team,
    x="region",
    y="Amount",
    color="Medal",
    labels={"Sport": "Sport", "0": "medals", "region": "Country"},
    barmode="group",
    title="Women's team cross country skiing medals by country",
    text_auto = True,
    color_discrete_sequence=[px.colors.qualitative.Dark2[6],px.colors.qualitative.Dark2[7],px.colors.qualitative.Dark2[5]]
)
fig.show()

___
## Total individual medals by country

In [18]:
total_individual_medals = reusable_df[reusable_df["Event"] != "Cross Country Skiing Men's Team Sprint"]

total_individual_medals = total_individual_medals[total_individual_medals["Event"] != "Cross Country Skiing Women's Team Sprint"]

total_individual_medals = total_individual_medals[total_individual_medals["Event"] != "Cross Country Skiing Women's 4 x 5 kilometres Relay"]

total_individual_medals = total_individual_medals[total_individual_medals["Event"] != "Cross Country Skiing Women's 3 x 5 kilometres Relay"]

total_individual_medals = total_individual_medals[total_individual_medals["Event"] != "Cross Country Skiing Men's 4 x 10 kilometres Relay"]

#print(total_individual_medals.to_string())
total_individual_medals.head()

,Sex,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
3969,M,Italy,ITA,1992 Winter,1992,Winter,Albertville,Cross Country Skiing,Cross Country Skiing Men's 10 kilometres,Silver,Italy,NaN
3973,M,Italy,ITA,1994 Winter,1994,Winter,Lillehammer,Cross Country Skiing,Cross Country Skiing Men's 10 kilometres,Bronze,Italy,NaN
5422,M,Norway,NOR,1994 Winter,1994,Winter,Lillehammer,Cross Country Skiing,Cross Country Skiing Men's 30 kilometres,Gold,Norway,NaN
5427,M,Norway,NOR,1998 Winter,1998,Winter,Nagano,Cross Country Skiing,Cross Country Skiing Men's 10/15 kilometres Pu...,Gold,Norway,NaN
5430,M,Norway,NOR,2002 Winter,2002,Winter,Salt Lake City,Cross Country Skiing,Cross Country Skiing Men's 10/10 kilometres Pu...,Gold,Norway,NaN


In [19]:
total_individual_medals = total_individual_medals.groupby(["Event", "region"])[["Medal"]].value_counts().to_frame().reset_index()


In [20]:
total_individual_medals = total_individual_medals.rename({0:'Amount'}, axis=1)

total_individual_medals.Medal = pd.Categorical(total_individual_medals.Medal,categories=['Bronze', 'Silver', 'Gold'])
total_individual_medals = total_individual_medals.sort_values('Medal')
total_individual_medals

,Event,region,Medal,Amount
174,Cross Country Skiing Women's Sprint,Slovenia,Bronze,2
125,Cross Country Skiing Women's 15 km Skiathlon,Poland,Bronze,1
55,Cross Country Skiing Men's 30 kilometres,Sweden,Bronze,1
126,Cross Country Skiing Women's 15 km Skiathlon,Russia,Bronze,1
52,Cross Country Skiing Men's 30 kilometres,Russia,Bronze,4
...,...,...,...,...
62,Cross Country Skiing Men's 30 km Skiathlon,Russia,Gold,1
116,Cross Country Skiing Women's 15 kilometres,Italy,Gold,2
64,Cross Country Skiing Men's 30 km Skiathlon,Sweden,Gold,1
56,Cross Country Skiing Men's 30 kilometres,Sweden,Gold,1


In [21]:
fig = px.histogram(
    total_individual_medals,
    x="region",
    y="Amount",
    color="Medal",
    labels={"Sport": "Sport", "0": "medals", "region": "Country"},
    barmode="group",
    title="Total individual medals by country",
    text_auto = True,
    color_discrete_sequence=[px.colors.qualitative.Dark2[6],px.colors.qualitative.Dark2[7],px.colors.qualitative.Dark2[5]]
)
fig.show()

In [23]:
xcs_df.head()
df_germany_2 = xcs_df.query("region == 'Germany'")
df_germany_2.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
5842,3288,Else Amann,F,32.0,NaN,NaN,Germany,GER,1956 Winter,1956,Winter,Cortina d'Ampezzo,Cross Country Skiing,Cross Country Skiing Women's 10 kilometres,NaN,Germany,NaN
5843,3288,Else Amann,F,32.0,NaN,NaN,Germany,GER,1956 Winter,1956,Winter,Cortina d'Ampezzo,Cross Country Skiing,Cross Country Skiing Women's 3 x 5 kilometres ...,NaN,Germany,NaN
7166,4005,Carola Anding (-Jacob),F,19.0,168.0,53.0,East Germany,GDR,1980 Winter,1980,Winter,Lake Placid,Cross Country Skiing,Cross Country Skiing Women's 10 kilometres,NaN,Germany,NaN
7167,4005,Carola Anding (-Jacob),F,19.0,168.0,53.0,East Germany,GDR,1980 Winter,1980,Winter,Lake Placid,Cross Country Skiing,Cross Country Skiing Women's 4 x 5 kilometres ...,Gold,Germany,NaN
7168,4005,Carola Anding (-Jacob),F,23.0,168.0,53.0,East Germany,GDR,1984 Winter,1984,Winter,Sarajevo,Cross Country Skiing,Cross Country Skiing Women's 5 kilometres,NaN,Germany,NaN


In [46]:
fig = px.bar(
    df_germany_2,
    x="Year",
    y="Year",
    color="Medal",
    labels={"Sport": "Sport", "0": "medals", "region": "Country"},
    barmode="group",
    title="Total individual medals by country",
    #text_auto = True,
    color_discrete_sequence=[px.colors.qualitative.Dark2[7],px.colors.qualitative.Dark2[5],px.colors.qualitative.Dark2[6]]
)
fig.show()